# Basic Examples

<!-- [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vitostamatti/discrete-event-simulation-simpy/blob/main/notebooks/basic_examples.ipynb) -->


In this set of examples, I'll explore the ``simpy`` library and it's main
objects. It's not the most intuitive library but not because of the design
or quality of it, but for the non trivial problem that it aims to solve.

Discrete Event Simulation with python is not an easy task, and I'm far from
being an expert in this matter, so I apologies in advance if a make a wrong
use of the simpy liabry or the concepts behind.

In [7]:
try:
    import simpy
except:
    !pip install simpy

## 01-Introductory example

First example is just to ilustrate how to start a simulation and
what simpy needs make its magic.

First of all, we need to import and install simpy. Then, to
make an event ocurr we need to use python generators.
The explanation of generators if far beyond this notebook so
I strongly recommend to take a look on to the [official python documentation](link)

The way I like to think about generators in this specific scenario
is that they will "freeze" the excecution until a condition is passed.
In our case, this conditions is the env.timeout() event, which internally
makes the simulation time to advance. After this timeout is finished, the
excecution continues from where it was interrupted.

In [4]:
import simpy


def example(env):
    # we are about to freeze excecution for 1 unit
    print('Before timeout: now=%d' % (env.now))
    value = yield env.timeout(1, value=42)
    # now we returned to the excecution and continue
    print('After timeout: now=%d, value=%d' % (env.now, value))

env = simpy.Environment()
p = env.process(example(env))
env.run()

Before timeout: now=0
After timeout: now=1, value=42


## 02-Python class example

An alternative to python functions, is to use python classes (which I personally prefer).
This allows us to build much more "interpretable" models where the simulation objects
represents real objects.

In [5]:
import simpy

# Second example
class Example():

    def __init__(self, env, delay=10):
        self.env = env
        self.delay = delay

    def process(self):
        print('Before timeout: now=%d' % (env.now))
        value = yield self.env.timeout(self.delay, value=42)
        print('After timeout: now=%d, value=%d' % (env.now, value))


env = simpy.Environment()
e = Example(env,delay=10)
env.process(e.process())
env.run()

Before timeout: now=0
After timeout: now=10, value=42


## 03-Vehicle example

In this example, we'll make a process to run uninterruptedly
and to let simpy enviroment finish the simulation when a maximum time
is reached. This kind of simulation it's mostly use to emulate
real time dependent process and to evaluate it's progress over time.


In [6]:
import simpy
import numpy as np

class Vehicle:
    """
    Vehicle object that can drive() for a given time
    depending on its fuel consumption and fuel capacity.


    Attributes:
        env (simpy.Environment): the simpy environment to run with.
        min_speed (float, optional): min speed expressed in km/hs. Defaults to 80.
        max_speed (float, optional): max speed expressed in km/hs. Defaults to 100.
        fuel_consumption (float, optional): fuel consumption expressed in km/lt. Defaults to 10.
        fuel_capacity (float, optional): fuel capacity expressed in lt. Defaults to 35.
    """
    def __init__(
            self, 
            env:simpy.Environment, 
            min_speed:float=80., 
            max_speed:float=100, 
            fuel_consumption:float=10, 
            fuel_capacity:float=35
            ):


        self.env = env
        self.min_speed = min_speed # km/hs
        self.max_speed = max_speed # km/hs
        self.fuel_consumption = fuel_consumption # km/lt
        self.fuel_capacity = fuel_capacity # lt
        self.fuel = fuel_capacity # lt

        
    def drive(self):
        """Excecutes the loop that makes the vehicle drive
        """
        while True:
            print(f"Start Driving at {self.env.now}")
            travel_time = (self.fuel*self.fuel_consumption)/(np.random.randint(self.min_speed,self.max_speed))
            yield self.env.timeout(travel_time)
            print(f"Need refueling at {self.env.now}")

            print(f"Start refueling at {self.env.now}")
            yield self.env.timeout(np.random.uniform(0.05, 0.15))
            self.fuel = np.random.randint(self.fuel_capacity-5,self.fuel_capacity)
            print(f"Finished refueling at {self.env.now}: fuel now is {self.fuel}")


env = simpy.Environment()
v = Vehicle(env)
env.process(v.drive())
env.run(until=24)

Start Driving at 0
Need refueling at 4.117647058823529
Start refueling at 4.117647058823529
Finished refueling at 4.201092763091564: fuel now is 31
Start Driving at 4.201092763091564
Need refueling at 7.891568953567755
Start refueling at 7.891568953567755
Finished refueling at 7.949270907409045: fuel now is 32
Start Driving at 7.949270907409045
Need refueling at 11.627431826949275
Start refueling at 11.627431826949275
Finished refueling at 11.719119299460319: fuel now is 31
Start Driving at 11.719119299460319
Need refueling at 15.282337690264917
Start refueling at 15.282337690264917
Finished refueling at 15.373105979395236: fuel now is 33
Start Driving at 15.373105979395236
Need refueling at 19.1662094276711
Start refueling at 19.1662094276711
Finished refueling at 19.28877342496866: fuel now is 34
Start Driving at 19.28877342496866
Need refueling at 23.025037161232397
Start refueling at 23.025037161232397
Finished refueling at 23.16057332216167: fuel now is 32
Start Driving at 23.1605

## 04-Machine Example

In this example, we simulate a machine that processes products in 
a queu or production plan. The policy used to select the next product
can be random, FIFO (first in, first out) of LIFO (last in, first out).

In [4]:
# Example with an entity being processed
import copy


class Product():
    def __init__(self, id):
        self.id = id
        self.processed = False


class Machine():
    def __init__(self, env:simpy.Environment, id, process_time=(10,15,20)):
        self.env = env
        self.id = id
        self.process_time = process_time
        self.idle = True 

    def generate_process_time(self):
        return np.random.triangular(
            self.process_time[0], 
            self.process_time[1], 
            self.process_time[2]
        )

    def process_product(self, product):
        print(f"Processing product {product.id} at time {self.env.now}")
        self.idle = False
        yield self.env.timeout(delay=self.generate_process_time())
        product.processed = True
        self.idle = False
        return product

    def _set_production_plan(self, products):
        self.pending_products = copy.deepcopy(products)
        self.processed_products = []

    def process_all_products_randomly(self,products):
        self._set_production_plan(products)
        while len(self.pending_products)>0:
            next_prod = np.random.choice(range(len(self.pending_products)))
            p = self.pending_products[next_prod]
            yield env.process(m.process_product(p))
            self.pending_products.remove(p)
            self.processed_products.append(p)

    def process_all_products_fifo(self,products):
        self._set_production_plan(products)
        while len(self.pending_products)>0:
            p = self.pending_products[0]
            yield env.process(m.process_product(p))
            self.pending_products.remove(p)
            self.processed_products.append(p)

    def process_all_products_lifo(self,products):
        self._set_production_plan(products)
        while len(self.pending_products)>0:
            p = self.pending_products[-1]
            yield env.process(m.process_product(p))
            self.pending_products.remove(p)
            self.processed_products.append(p)



print("Processing RANDOM policy")
env = simpy.Environment()
m = Machine(env,id='machine')
products = [Product(id=i) for i in range(10)]
env.process(m.process_all_products_randomly(products))
env.run()
print()

print("Processing FIFO policy")
env = simpy.Environment()
m = Machine(env,id='machine')
products = [Product(id=i) for i in range(10)]
env.process(m.process_all_products_fifo(products))
env.run()
print()

print("Processing LIFO policy")
env = simpy.Environment()
m = Machine(env,id='machine')
products = [Product(id=i) for i in range(10)]
env.process(m.process_all_products_lifo(products))
env.run()

Processing RANDOM policy
Processing product 5 at time 0
Processing product 6 at time 12.146334706420241
Processing product 9 at time 25.41922320679302
Processing product 4 at time 40.41212099773223
Processing product 1 at time 56.588785123494546
Processing product 3 at time 72.90007562837951
Processing product 0 at time 86.80018851221942
Processing product 2 at time 100.05120682763642
Processing product 7 at time 118.10001346524449
Processing product 8 at time 132.93144246633773

Processing FIFO policy
Processing product 0 at time 0
Processing product 1 at time 16.536097773367587
Processing product 2 at time 33.489174435881715
Processing product 3 at time 46.470758671131634
Processing product 4 at time 61.69361932247431
Processing product 5 at time 79.3962876115119
Processing product 6 at time 97.03127374365525
Processing product 7 at time 110.8215196891729
Processing product 8 at time 126.91600354007943
Processing product 9 at time 145.3705580020133

Processing LIFO policy
Processing 